In [1]:
# Install the LiteLLM library, which lets us call different AI models (like OpenAI, Groq, Gemini) using one simple interface
# We run this in Colab using !pip to install the package
# LiteLLM is essential for this project because it simplifies API calls and lets us switch models easily
!pip install litellm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 30.8 MB/s eta 0:00:00


In [2]:
# Import libraries needed for the project
import os        # Used to set environment variables (like API keys)
import time     # Used to measure how fast each model responds
from litellm import completion  # The main LiteLLM function to call AI models
from google.colab import userdata  # Used to access API keys securely in Colab

# Set up API keys securely using Colab Secrets
# This avoids hardcoding sensitive keys in the code, keeping them safe
# LiteLLM needs these keys to authenticate with OpenAI, Groq, and Gemini
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_')  # Get OpenAI key from Colab Secrets
    os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')      # Get Groq key
    os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')  # Get Gemini key
except Exception as e:
    print("Error: Please set API keys in Colab Secrets (OPENAI_API_KEY, GROQ_API_KEY, GEMINI_API_KEY).")
    print("Go to the key icon in the sidebar, add each key, and rerun this cell.")
    raise e  # Stop execution if keys are missing

# Print a confirmation to ensure keys are set
print("API keys loaded successfully!")

API keys loaded successfully!


In [9]:
# Define simulated cow data to mimic real sensor data from a dairy farm
# This data represents one cow's health and production metrics
# In a real farm, this data would come from sensors (e.g., collars, milking machines)
cow_data = {
    "cow_id": "COW123",
    "rumination_time": 400,      # Minutes/day the cow chews cud (normal: 450-550)
    "activity_level": 500,       # Steps/day the cow takes (normal: 600-800)
    "milk_yield_yesterday": 25,  # Liters of milk produced yesterday
    "udder_temperature": 39.5,   # °C, udder temperature (normal: 38-39)
    "feed_intake": 20            # kg/day of feed consumed
}

# Define the AI models we’ll test
# LiteLLM lets us use these models with the same code, even though they’re from different providers
models = [
    "openai/gpt-4o-mini",              # OpenAI’s GPT-4o: great for detailed reasoning
    "groq/llama-3.3-70b-versatile", # Groq’s LLaMA: fast for real-time predictions
    "gemini/gemini-1.5-flash"     # Gemini: cost-effective, supports images (if needed later)
]

# Print the cow data to confirm what we’re analyzing
print("Cow Data for Analysis:")
for key, value in cow_data.items():
    print(f"{key}: {value}")

Cow Data for Analysis:
cow_id: COW123
rumination_time: 400
activity_level: 500
milk_yield_yesterday: 25
udder_temperature: 39.5
feed_intake: 20


In [10]:
# Function to detect cow health issues using LiteLLM
# This sends the cow data to each AI model and asks it to identify problems like mastitis or lameness
def detect_health_issues(cow_data, model):
    # Create a detailed prompt to guide the AI
    # The prompt includes the cow data, normal ranges, and instructions for a concise response
    prompt = f"""
    You are a dairy farm AI assistant with expertise in veterinary science.
    Analyze the following cow data to identify potential health issues (e.g., mastitis, lameness).
    Provide a one-sentence diagnosis and a one-sentence recommendation.
    Use the following data:
    - Cow ID: {cow_data['cow_id']}
    - Rumination Time: {cow_data['rumination_time']} minutes/day (normal: 450-550)
    - Activity Level: {cow_data['activity_level']} steps/day (normal: 600-800)
    - Udder Temperature: {cow_data['udder_temperature']}°C (normal: 38-39)
    - Milk Yield Yesterday: {cow_data['milk_yield_yesterday']} liters
    - Feed Intake: {cow_data['feed_intake']} kg/day
    Example response: "The cow may have mastitis due to elevated udder temperature; recommend a veterinary exam."
    """
    # Call the AI model using LiteLLM’s completion function
    # temperature=0.3 ensures consistent, factual responses
    # max_tokens=100 keeps the response short
    try:
        response = completion(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=100
        )
        # Return the AI’s response (the diagnosis and recommendation)
        return response.choices[0].message.content.strip()
    except Exception as e:
        # If the API call fails (e.g., rate limit, wrong key), return an error message
        return f"Error with {model}: {str(e)}"

# Test the function with one model to ensure it works
test_model = models[0]  # Use the first model (GPT-4o)
print(f"Testing health diagnosis with {test_model}...")
health_result = detect_health_issues(cow_data, test_model)
print(f"Result: {health_result}")

Testing health diagnosis with openai/gpt-4o-mini...
Result: "The cow may be experiencing health issues related to reduced rumination time and activity level, along with elevated udder temperature, indicating potential mastitis; recommend a veterinary exam and monitoring of her overall health."


In [11]:
# Function to predict today’s milk yield using LiteLLM
# This asks the AI to estimate milk production based on cow data
def predict_milk_yield(cow_data, model):
    # Create a prompt to predict milk yield
    # It includes all cow data and asks for a prediction plus an explanation
    prompt = f"""
    You are a dairy farm AI assistant specializing in milk production.
    Predict today's milk yield (in liters) for the cow based on the following data.
    Provide a one-sentence prediction and a one-sentence explanation.
    Use the following data:
    - Cow ID: {cow_data['cow_id']}
    - Rumination Time: {cow_data['rumination_time']} minutes/day
    - Activity Level: {cow_data['activity_level']} steps/day
    - Udder Temperature: {cow_data['udder_temperature']}°C
    - Milk Yield Yesterday: {cow_data['milk_yield_yesterday']} liters
    - Feed Intake: {cow_data['feed_intake']} kg/day
    Example response: "Predicted milk yield is 22 liters; lower rumination and high udder temperature suggest health stress."
    """
    # Call the AI model using LiteLLM
    # temperature=0.5 allows slight variation in predictions
    try:
        response = completion(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=100
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error with {model}: {str(e)}"

# Test the function with one model
test_model = models[0]  # Use GPT-4o
print(f"Testing milk yield prediction with {test_model}...")
yield_result = predict_milk_yield(cow_data, test_model)
print(f"Result: {yield_result}")

Testing milk yield prediction with openai/gpt-4o-mini...
Result: Predicted milk yield is 23 liters; the high rumination time and adequate feed intake indicate good health and potential for stable production.


In [12]:
# Process the cow data using all models and compare their results
# This cell runs both functions for each model and measures performance
results = {}  # Store results for each model
print(f"Analyzing cow {cow_data['cow_id']} with all models...\n")

# Loop through each model (OpenAI, Groq, Gemini)
for model in models:
    # Record the start time to measure how fast the model responds
    start_time = time.time()

    # Get health diagnosis
    health_diagnosis = detect_health_issues(cow_data, model)

    # Get milk yield prediction
    yield_prediction = predict_milk_yield(cow_data, model)

    # Calculate time taken for both tasks
    elapsed_time = time.time() - start_time

    # Store results in a dictionary
    results[model] = {
        "health_diagnosis": health_diagnosis,
        "yield_prediction": yield_prediction,
        "time_taken": elapsed_time
    }

# Display results for all models
print("Comparison of Model Results:")
for model, result in results.items():
    print(f"\nModel: {model}")
    print(f"Health Diagnosis: {result['health_diagnosis']}")
    print(f"Milk Yield Prediction: {result['yield_prediction']}")
    print(f"Time Taken: {result['time_taken']:.2f} seconds")
    print("-" * 50)

Analyzing cow COW123 with all models...


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

Comparison of Model Results:

Model: openai/gpt-4o-mini
Health Diagnosis: The cow may be experiencing health issues related to mastitis due to elevated udder temperature and decreased rumination and activity levels; recommend a veterinary exam and monitoring of milk quality.
Milk Yield Prediction: Predicted milk yield is 28 liters; high rumination time and adequate feed intake indicate good health and potential for increased milk production.
Time Taken: 2.15 seconds
--------------------------------------------------

Model: groq/llama-

In [13]:
# Test the system with different cow data scenarios
# This helps us see how the AI handles healthy vs. unhealthy cows
test_cases = [
    {
        "cow_id": "COW456",
        "rumination_time": 500,      # Normal
        "activity_level": 700,       # Normal
        "milk_yield_yesterday": 28,  # Good yield
        "udder_temperature": 38.5,   # Normal
        "feed_intake": 22            # Normal
    },
    {
        "cow_id": "COW789",
        "rumination_time": 300,      # Very low
        "activity_level": 400,       # Low
        "milk_yield_yesterday": 20,  # Low yield
        "udder_temperature": 40.0,   # Very high
        "feed_intake": 18            # Low
    }
]

# Process each test case
for test_data in test_cases:
    print(f"\nTesting Cow {test_data['cow_id']}...")
    results = {}
    for model in models:
        start_time = time.time()
        health_diagnosis = detect_health_issues(test_data, model)
        yield_prediction = predict_milk_yield(test_data, model)
        elapsed_time = time.time() - start_time
        results[model] = {
            "health_diagnosis": health_diagnosis,
            "yield_prediction": yield_prediction,
            "time_taken": elapsed_time
        }

    # Display results
    print(f"Results for Cow {test_data['cow_id']}:")
    for model, result in results.items():
        print(f"\nModel: {model}")
        print(f"Health Diagnosis: {result['health_diagnosis']}")
        print(f"Milk Yield Prediction: {result['yield_prediction']}")
        print(f"Time Taken: {result['time_taken']:.2f} seconds")
        print("-" * 50)


Testing Cow COW456...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

Results for Cow COW456:

Model: openai/gpt-4o-mini
Health Diagnosis: The cow shows no immediate signs of health issues as all parameters are within normal ranges; however, continue to monitor for any changes in milk yield or behavior, and ensure regular veterinary check-ups.
Milk Yield Prediction: Predicted milk yield is 30 liters; the high rumination time and adequate feed intake indicate good digestive health, likely contributing to increased milk production.
Time Taken: 3.54 seconds
--------------------------------------------------

Model: groq/llama-3.3-70b-versatile
Health Diagnosis: Error with groq/llama-3.3-70b-versatile: litellm.InternalServerError: InternalServerError: GroqException - {"error":{"message":"Internal Server Error","type":"internal_ser